# Dementia and Alzheimers: Early Predictors

## Dataset

The data used in this project comes from the OASIS (Open Access Series of Imaging Studies) Brains organization, which aims to provide neuroscientific images and data freely to the scientific community. The citations for both datasets are below:

OASIS: Cross-Sectional: Principal Investigators: D. Marcus, R, Buckner, J, Csernansky J. Morris; P50 AG05681, P01 AG03991, P01 AG026276, R01 AG021910, P20 MH071616, U24 RR021382


OASIS: Longitudinal: Principal Investigators: D. Marcus, R, Buckner, J. Csernansky, J. Morris; P50 AG05681, P01 AG03991, P01 AG026276, R01 AG021910, P20 MH071616, U24 RR021382

This dataset was accessed through Kaggle at https://www.kaggle.com/jboysen/mri-and-alzheimers



## Motivation 

Dementia and its specific subtype of Alzheimer's disease are some of the most pervasive health conditions that affect millions of people around the world. A diagnosis of this disease is heartbreaking not only for the patient but also for their families, because this means that over time, the patient will slowly lose touch with the world around them and might even stop recognizing their loved ones. 

Brain scans of Alzheimer's and dementia patients indicate loss of white matter volume, deteriorated neuronal connections, and a buildup of amyloid plaques. Taken all together, the patients exhibit symptoms such as short-term memory loss, general forgetfulness, increasing dependence on caregivers, speech loss, and other coordination problems.

Although a drug called Aduhelm was recently approved to be fast-tracked by the FDA as a treatment for Alzheimer's disease (https://www.fda.gov/news-events/press-announcements/fda-grants-accelerated-approval-alzheimers-drug), the evidence for the effectiveness of the drug is not overwhelmingly positive, and at best, the drug will only slow down the deterioration. 

Given the widespread nature of the disease and the burden that it places on society as a whole, studying this disease and 